**Para essa atividade baixe os dois samples de audio do repositório para testar, ou use seu próprio sample**


Primeiramente instalamos todas as bibliotecas necessarias para essa atividade:
- whisper - OpenAI
- SimpleAudio
- Playsound
- Deep Translator - Google
- TxtAI

In [1]:
! pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [9]:
pip install simpleaudio


In [10]:
pip install playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=d5731beca1be103245ca1d3f0d270186312c5a73afc536ce5d90115613d1f5d6
  Stored in directory: /root/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound


In [35]:
pip install deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.0 MB/s eta 0:00:00


In [2]:
pip install txtai[pipeline-audio] soundfile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.3/198.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 104.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.7 MB/s eta 0:00:00


Aqui importamos as libs


In [36]:
import os
import whisper

import soundfile as sf
import simpleaudio as sa
from transformers import pipeline

from txtai.pipeline import TextToSpeech
from deep_translator import GoogleTranslator

import time

import playsound

Carregamos o modelo 'base' do whisper que será o suficiente para nossa atividade

In [12]:
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 58.9MiB/s]


Aqui apenas verificamos caso o colab esteja rodando com a GPU, isso é necessario para o Whisper

Caso o 'type' não seja 'cuda', MUDAR ISSO EM Runtime->Change Runtime Type

In [13]:
model.device

device(type='cuda', index=0)

Criação da **Classe de Text To Speech**, sua unica função é de gerar o audio com base no input de textp que passamos para ela em _generate_audio_, ela nos retorna o caminho onde salvamos o arquivo, aqui no caso '/content/out.wav'

In [14]:

class TTS():
  def __init__(self):
        pass

  def generate_audio(self, text):
    # Build pipeline
    tts = TextToSpeech("NeuML/ljspeech-jets-onnx")

    # Generate speech
    speech = tts(text)

    # Write to file
    sf.write("out.wav", speech, 22050)

    # Read the file
    wave_obj = sa.WaveObject.from_wave_file("/content/out.wav")

    path_audio = '/content/out.wav'
    return path_audio



Criação da classe **Speech To Text**, nela carregamos o modelo do whisper e sua função _transcribe_audio_ le o arquivo que passamos como parametro e retorna a transcrição dele.


In [28]:
class STT():
  def __init__(self):
      self.model = whisper.load_model("base")

  def transcribe_audio(self, audio_path):
      model_result = self.model.transcribe(audio_path)
      transcription = model_result["text"]
      return transcription

**Execução do programa**

Essa celula ira rodar o speech-to-text no arquivo que foi passado como parametro na variavel _text_stt_, apos isso a string sera printada.
Com a transcrição, esse texto é passado para o tradutor da biblioteca do google, traduzindo do ingles para o portugues.
Logo após isso o texto é passado como parametro para a funçõa _generate_audio_ dentro da classe de text-to-speech. O modelo usado é treinaod e em ingles e o audio pode ficar com muito sotaque, traduzir um audip do portugues par ao ingles pode ser uma melhor opção

In [40]:

stt = STT()
tts = TTS()
print("Ouvindo o audio ")

text_stt = stt.transcribe_audio('/content/jackhammer.wav')
print(f'O audio transcrito é: {text_stt}')
time.sleep(2)

translator = GoogleTranslator(source='en', target='pt')
translation = translator.translate(text=text_stt)
print(f'O texto traduzido para pt-BR: {translation}')
time.sleep(2)


speech_translated = tts.generate_audio(translation)
time.sleep(2)



Ouvindo o audio 
O audio transcrito é:  The still smell of cold beer lingers.
O texto traduzido para pt-BR: O cheiro ainda de cerveja gelada permanece.
Escute o audio traduzido


CalledProcessError: ignored

Abaixo tocamos o audio traduzido dentro do Google Colab

In [42]:
from IPython.display import Audio
wn = Audio(speech_translated, autoplay=True)
display(wn)